In [1]:
#Load json files from the web and convert bytes into utf

In [26]:
import re
import urllib  # the lib that handles the url stuff
import json

def getJsonFromWeb(url):
    data = urllib.request.urlopen(url)
    raw_data = data.read()
    encoding = data.info().get_content_charset('utf8')  # JSON default
    return raw_data.decode(encoding)

def convertKeyNames(json):
    #Convert the strings into actual JSON objects - first regex fixes the keys, second the values
    json = re.sub("((?=\D)\w+):", r'"\1":',  json)
    json = re.sub(": ((?=\D)\w+)", r':"\1"',  json)
    return json

def getOnMapJsonFromWeb(url, index):
    data = getJsonFromWeb(url)
    #Take out the variable names to get pure json format
    map_json = str(data.split("map_json["+ str(index) + "] = ")[1])
    map_json = str(map_json.split(";")[0])
    on_map_json = str(data.split("on_map_json[" + str(index) + "] = ")[1])
    on_map_json = str(on_map_json.split(";")[0]) 
    on_map_json = convertKeyNames(on_map_json)
    return on_map_json

def loadBaseFiles(path, filterStr):
    data = ""
    with open(path, 'r') as data_file:
        data = data_file.read()

    if (filterStr!= None):
        data = str(data.split(filterStr)[1])
    data = str(data.split(";")[0])
        
    return convertKeyNames(data);

def loadBaseFilesFromUrl(url, filterStr):
    #data = getJsonFromWeb(url)
    data = str(data.split(filterStr)[1])
    data = str(data.split(";")[0])
    return convertKeyNames(data)



In [56]:
import time
#ENTER THE NUMBER OF MAPS FROM RPG MO
map_number = 43

#On_map of all maps will be directly accesible here, maps[mapId][index] returns tile value
maps = {}

#ime out sleep to avoid smapping the server... for some reason data.mo.ee throws timeouts 
for mapId in range(0, map_number):
    maps[mapId] = json.loads(getOnMapJsonFromWeb("http://rpg.mo.ee/maps/map"+str(mapId)+".js", mapId));
    time.sleep(1)


In [57]:
#read object base https://www.dropbox.com/s/si3xwwl0qeox8p0/object_base.js?dl=0
#data = urllib.request.urlopen("https://dl.dropboxusercontent.com/s/si3xwwl0qeox8p0/object_base.js")
#raw_data = data.read()
#encoding = data.info().get_content_charset('utf8')  # JSON default
#data = raw_data.decode(encoding)

#object_base = data.split("var object_base = ")[1]
#object_base = object_base.split(";")[0]
#object_base = json.loads(object_base)


# parse file
#npc_base = json.loads(loadBaseFilesFromUrl("https://dl.dropboxusercontent.com/s/yjj3c8mdfw3h6zx/npc_base.js", "var npc_base = "))
npc_base = loadBaseFiles("./npc_base.js", "var npc_base = ")
npc_base = json.loads(npc_base);

In [59]:
#Compute tiles for Monsters
computed_tiles = {}
for mapId in range(0, map_number):
    computed_tiles[mapId] = []
    for tile in maps[mapId]:
        #Filter out unnecessary
        if(tile['b_t'] == "4"):
            for prop in npc_base[tile['b_i']]['activities']:
                if ("Attack" in prop):
                    computed_tiles[mapId].append([tile['i'], tile['j'], tile['b_i']])

In [60]:
#The affinity propagation is suboptimal in some cases, eg. bosses get grouped with other mobs :| 
#At this moment I dont know how to make different b_i mutally exclusive, 
#e.g. a lone mob should not be groupped with other Mobs...
#To overcome this issue we can first sort the map for each b_i and then run Affinity Propagation for each bi
#There is probably a better way to do this, but it works so...

#UPDATE: 
#Well i handel it later as followe
#When there is only one mob there is no cluster, so lets use it
#If only a single entity, and not a boss, lets skip it since that cluster is not important.

In [61]:
#First need to rearrange the map tiles
filtered_tiles = {}
for mapId in computed_tiles:
    filtered_tiles[mapId] = {}
    for tile in computed_tiles[mapId]:
        #if b_i does not exist create array
        if tile[2] not in filtered_tiles[mapId] :
            filtered_tiles[mapId][tile[2]] = []
        #... and append
        filtered_tiles[mapId] [tile[2]].append([tile[0], tile[1]]);

#Print dorpat's monsters and tiles
print(filtered_tiles[0])

{101: [[6, 32], [6, 38], [7, 34], [8, 27], [9, 31], [11, 27], [14, 30], [15, 27], [18, 29]], 0: [[7, 49], [9, 46], [9, 54], [9, 57], [10, 51], [13, 46], [13, 54], [13, 61], [14, 58], [17, 51], [17, 57], [18, 44], [18, 48], [21, 44], [26, 44], [26, 50], [30, 47], [41, 32], [44, 28], [44, 32], [45, 22], [47, 30], [48, 23], [50, 26], [51, 30]], 1: [[8, 41], [24, 36], [24, 39], [25, 31], [27, 39], [28, 33], [29, 41], [31, 33], [33, 27], [36, 38], [37, 22], [37, 34], [38, 30], [39, 26], [75, 81], [76, 84], [77, 77], [78, 73], [81, 74], [84, 72], [88, 72]], 100: [[9, 36], [10, 39], [12, 34], [13, 36], [13, 39], [14, 33], [15, 37], [16, 34], [17, 11], [19, 31], [19, 33]], 13: [[12, 81], [13, 77], [14, 79], [16, 76], [16, 80], [57, 91], [61, 93], [62, 89], [68, 93], [76, 62], [78, 56], [79, 60], [80, 53], [83, 56], [83, 63], [84, 51], [86, 61], [87, 51], [89, 57]], 6: [[13, 88], [13, 91], [15, 89], [15, 92], [16, 87], [18, 90], [20, 88], [20, 91], [22, 86], [22, 89], [22, 92], [24, 89], [25, 9

In [63]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
import numpy as np

#We want to compute affinity of the monsters on a map individually and
# combine it back before rendering the graph on the screen

centers = {}

def ComputeAffinity(mapId):
    
    temp_cluster_centers_indices = {}
    temp_labels = {}
    
    for monster_id in filtered_tiles[mapId]:
        
        X = np.array(filtered_tiles[mapId][monster_id])
        
        # Compute Affinity Propagation
        af = AffinityPropagation(preference=-2005).fit(X)
        cluster_centers_indices = af.cluster_centers_indices_
        labels = af.labels_
        
        #When there is only one mob there is no cluster, so lets use it
        if(af.cluster_centers_indices_ is None):
            if("BOSS" in npc_base[monster_id]["name"] ):
                #print("BOSS")
                cluster_centers_indices = [0]
                labels = np.array([0, 0])
            #If only a single entity, and not a boss, lets skip it since that cluster is not important.
            else:
                #print("Single Mob")
                continue;
                
        #We want to combine the results together, hence we write:
        temp_cluster_centers_indices[monster_id] = cluster_centers_indices
        temp_labels[monster_id] = labels
                
    
    #Concatenate the labels
    #cluster_centers_indices = np.concatenate([temp_cluster_centers_indices[x] for x in temp_cluster_centers_indices])
    #labels = np.concatenate([temp_labels[x] for x in temp_labels])
        
    
    
    
    
    
    
    #print("X length", len(X))
    #print('Estimated number of clusters: %d' % n_clusters_)
    #print("Silhouette Coefficient: %0.3f"
    #      % metrics.silhouette_score(X, labels, metric='sqeuclidean'))

    #print(labels, cluster_centers_indices, X)
    
    # #############################################################################
    # Plot result
    #import matplotlib.pyplot as plt
    #from itertools import cycle

    #plt.close('all')
    #plt.figure(1)
    #plt.clf()

    #colors = cycle('bgrycmyk')
    #for monster in filtered_tiles[mapId]:
        #n_clusters_ = len(temp_cluster_centers_indices)
        #print("n_clusters_", n_clusters_, temp_cluster_centers_indices)
        #for k, col in zip(range(n_clusters_), colors):
            #class_members = temp_labels[monster_id] == k
            #print(class_members, temp_cluster_centers_indices[monster][k])
            #cluster_center = filtered_tiles[mapId][monster][temp_cluster_centers_indices[monster][k]]
            #print("cluster_center", cluster_center, class_members)

            #if n_clusters_ > 1:
            #print("T", filtered_tiles[mapId][monster][class_members, 0], filtered_tiles[mapId][monster][class_members, 1])
            #plt.plot(filtered_tiles[mapId][monster][class_members, 0], filtered_tiles[mapId][monster][class_members, 1], col + '.')
            #plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
            #         markeredgecolor='k', markersize=14)
            #if n_clusters_ > 1:
            #for x in filtered_tiles[mapId][monster][class_members]:
            #    plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col, label="red")

    #print x coordinates, y coordinates, 
    cluster_centers_indices = [];
        
    return temp_cluster_centers_indices;
    #plt.title('Estimated number of clusters: %d' % n_clusters_)
    #plt.show()

In [64]:
map_centers = {}
for mapId in range(0, map_number):
    #Tavern/Minigames has no mobs, which breaks the computation, so skip it
    if(mapId == 40 or mapId == 19 or mapId == 23):
        continue
    
    ##
    map_monster_centers = ComputeAffinity(mapId)
    
    #turn centers into coordinates + mob id

    map_centers[mapId] = []
        
    if(mapId == 40 or mapId == 19 or mapId == 23):
        continue;
        
    for monster in map_monster_centers:
        for center in map_monster_centers[monster]:
            coord = {}
            coord["x"] = filtered_tiles[mapId][monster][center][0]
            coord["y"] = filtered_tiles[mapId][monster][center][1]
            coord["b_i"] = monster
            map_centers[mapId].append(coord)
        
    
        


c:\Users\Administrator\anaconda3\envs\Atari\lib\site-packages\sklearn\cluster\_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\Atari\lib\site-packages\sklearn\cluster\_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\Atari\lib\site-packages\sklearn\cluster\_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\Atari\lib\site-packages\sklearn\cluster\_affinity_propagation.py:144: UserWarning: All samples have mutually equal similarities. Returning arbitrary cluster center(s).
  warnings.warn(
c:\Users\Administrator\anaconda3\envs\Atari\lib\site-packages\sklearn\cluster\_affinity_propagation.py:144: 

In [65]:
json.dumps(map_centers)


'{"0": [{"x": 9, "y": 31, "b_i": 101}, {"x": 17, "y": 51, "b_i": 0}, {"x": 44, "y": 28, "b_i": 0}, {"x": 31, "y": 33, "b_i": 1}, {"x": 81, "y": 74, "b_i": 1}, {"x": 14, "y": 33, "b_i": 100}, {"x": 14, "y": 79, "b_i": 13}, {"x": 61, "y": 93, "b_i": 13}, {"x": 83, "y": 56, "b_i": 13}, {"x": 18, "y": 90, "b_i": 6}, {"x": 28, "y": 62, "b_i": 3}, {"x": 52, "y": 43, "b_i": 3}, {"x": 22, "y": 80, "b_i": 4}, {"x": 65, "y": 89, "b_i": 4}, {"x": 77, "y": 20, "b_i": 4}, {"x": 31, "y": 80, "b_i": 120}, {"x": 89, "y": 38, "b_i": 120}, {"x": 42, "y": 13, "b_i": 102}, {"x": 47, "y": 60, "b_i": 187}, {"x": 43, "y": 58, "b_i": 201}, {"x": 52, "y": 76, "b_i": 119}, {"x": 52, "y": 17, "b_i": 280}, {"x": 47, "y": 64, "b_i": 8}, {"x": 50, "y": 63, "b_i": 25}, {"x": 63, "y": 91, "b_i": 807}, {"x": 65, "y": 52, "b_i": 7}, {"x": 65, "y": 75, "b_i": 62}, {"x": 70, "y": 76, "b_i": 60}, {"x": 79, "y": 5, "b_i": 185}, {"x": 71, "y": 72, "b_i": 162}, {"x": 89, "y": 81, "b_i": 722}, {"x": 82, "y": 41, "b_i": 693}],